In [7]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [17]:
#Load Neccessary for web scraping
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

from selenium.webdriver.common.by import By
#driver.get("https://cec.gmu.edu/about/meet-our-faculty?page=0")

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:01<00:00, 3.75MB/s]


In [18]:
#Collects the name and profile link for each person on the CEC faculty list
profiles_final = []
names = []
for i in range(0,25):
    #Sleep to avoide bot detection
    time.sleep(3)
    #Load each new page since each page only holds 10 names
    driver.get(f"https://cec.gmu.edu/about/meet-our-faculty?page={i}")
    src = driver.page_source  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'html.parser')
    soup = soup.find("ul", {"class": "profile-list"})
    urls = []
    # traverse paragraphs from soup
    for link in soup.find_all("a"):
        data = link.get('href')
        name = link.get_text()
        #Add name and link to lists
        urls.append(data)
        names.append(name)

    mystring = 'https://cec.gmu.edu'
    profiles_final = profiles_final + [mystring + s for s in urls]

In [19]:
#Create dataframe of names with profile links and save to csv
dict = {'Names': names, 'profile_urls': profiles_final} 
       
df = pd.DataFrame(dict)
df.to_csv('profiles_final.csv')

In [5]:
#Load if not running previous chunks
profiles = pd.read_csv('profiles_final.csv')
profiles_final = profiles['profile_urls']


In [20]:
#Create lists for each seperate field collected
i = 0
phone_list = []
title_list = []
bio_list = []
research_list = []
degree_list = []
link_list = []

#traverse through profile link file
for i in range(0,len(profiles_final)):
    #Pause to avoid bot detection
    time.sleep(3)
    #Load webpage
    driver.get(profiles_final[i])
    src = driver.page_source  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'html.parser')
    
    #Collect phone number
    person_info = soup.find_all("span", {"class": "info-staff"})
    phone = None
    for x in person_info:
        if x:
            x = x.get_text()
            if "Phone" in x:
                phone = x.replace('Phone: ', '')
                phone = phone.replace('(', '')
                phone = phone.replace(') ', '-')
    
    if phone:
        print(phone)
        phone_list.append(phone)
    else:
        phone_list.append("N/A")
    
    #Collect Job Title
    person_title = soup.find("div", {"class": "field field--name-field-org-positions field--type-text-long field--label-visually_hidden"})
    if person_title:
        person_title1 = person_title.find("p")
        title = person_title1.get_text()
        
    else:
        title = "N/A"
        
    title_list.append(title)
    
    #Collect Biography and Research
    person_bio = soup.find("div", {"class": "layout__region region-second"})
    if person_bio:
        person_degree = person_bio.find_all("li")
        temp = person_bio.get_text().split('Research\n')
        bio = temp[0].replace('Biography','')
        bio = bio.split('Degrees\n')[0]
        bio = bio.split('Honors and Awards\n')[0]
        bio = bio.replace("\n\n",'')
        #print(bio)
        if len(temp)>1:
            if "Research Interests" in temp[1]:
                res = temp[1].split("Degrees\n")[0].split("Research Interests\n")[-1]
                print(res)
            else:
                res = "N/A"
        else:
            res = "N/A"

        #Collect degrees
        degrees = ""
        for t in person_bio.find_all("li"):
            degrees = degrees + t.get_text() + "\n"
    else:
        bio = "N/A"
        degrees = "N/A"
    bio_list.append(bio)
    research_list.append(res)
    degree_list.append(degrees)
    
    
    # Personal links and email addresses
    content = soup.find("section", {"id": "content"})
    if content:
        links = ""
        for link in content.find_all('a'):
            links = links + link.get('href') + "\n"
    else:
        links = "N/A"
        
    link_list.append(links)
    
    
    i+=1

703-993-1511
Machine Learning

703-993-7155
703-993-1110
Artificial Intelligence, Computer Game Technology, and Computer Graphics

703-993-6158
703-993-1660
Software Engineering

703-993-5429
703-993-4383
703-993-5426
703-993-3786
Computer Systems, Parallel and Distributed Systems, Real-Time and Embedded Systems, Power-Aware Computing, Fault Tolerance

703-993-1674
703-993-5920
703-993-6148
703-993-5313
703-993-1500
Mechanical Engineering

703-993-1627
Databases
Honors and Awards
Outstanding Researcher, CS Department,  George Mason University

703-993-5638
703-993-1606
Electronics

703-993-4381
703-993-1591
703-993-6134
Research Interests:
Blast analysis and design of steel structural walls and roof systems
Improved blast resistance
Propagation of blast wave over roofs
Publications:
Bondok, D., Clayton, G., and Salim, H. (2018). “Dynamic Response of Roof Truss Systems under Blast Loading,” Structure Congress 2018
Bondok, D., and Salim, H. (2017). “Failure Capacities of Cold-Formed Stee

In [67]:
#Create Dataframe using lists storing collected inforamtion then save to a CSV file.
CEC = {'Name': profiles['Names'], 'Title': title_list, 'Phone Number': phone_list, 'Biography': bio_list, 
       'Research Interest': research_list, 'Degrees': degree_list, 'Links': link_list, 'Profile Link': profiles_final} 
       
df = pd.DataFrame(CEC)
df.fillna("N/A",inplace=True)
df['Biography'] = df['Biography'].replace(r'\n',' ', regex=True)
df['Research Interest'] = df['Research Interest'].replace(r'\n',',', regex=True)

In [68]:
#Remove errors in dataframe do to inconsistency in webpage format
for i in df['Research Interest']:
    if "Honors" in i:
        print(i)
        i = i.split("Honors")[0]
        print(i)
    if "Publications" in i:
        print(i)
        i = i.split("Publications")[0]
        print(i)

Databases,Honors and Awards,Outstanding Researcher, CS Department,  George Mason University,
Databases,
Research Interests:,Blast analysis and design of steel structural walls and roof systems,Improved blast resistance,Propagation of blast wave over roofs,Publications:,Bondok, D., Clayton, G., and Salim, H. (2018). “Dynamic Response of Roof Truss Systems under Blast Loading,” Structure Congress 2018,Bondok, D., and Salim, H. (2017). “Failure Capacities of Cold-Formed Steel Roof Trusses End-Connections,” Thin-Walled Structures Journal.,Bondok, D., Salim, H., and McClendon, M. (2017). “Static Resistance of Open Web Steel Joist Roof Systems for Blast Response Analysis,” ASCE Journal of Performance of Constructed Facilities, American Society of Civil Engineering (ASCE).,Bondok, D., Salim, H., Saucier, A., and Dinan, R. (2015). "Static Resistance Function of Cold-Formed Steel Stud Walls," Thin-Walled Structures Journal,Bondok, D., and Salim, H. (2014). “Numerical Modeling of Conventional St

In [69]:
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Links,Profile Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",,https://cec.gmu.edu/profiles/scarabal
1,Omoche Cheche Agada,Assistant Professor / Teaching,N/A,Omoche Cheche Agada is an Assistant Professor...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046\n,https://cec.gmu.edu/node/8046
2,Pouyan Ahmadi,Assistant Professor / Teaching,703-993-7155,Pouyan Ahmadi is an Assistant Professor in the...,N/A,"PhD, Electrical and Computer Engineering, Geo...",,https://cec.gmu.edu/profiles/pahmadi
3,Massimiliano Albanese,Associate Professor / Tenured\nAssociate Chair...,N/A,I am an Associate Professor in the Department...,N/A,"PhD, Computer Science & Engineering, Universi...",https://csis.gmu.edu/albanese/index.php\n,https://cec.gmu.edu/profiles/malbanes
4,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",http://cs.gmu.edu/~jallbeck/\n,https://cec.gmu.edu/profiles/jallbeck
...,...,...,...,...,...,...,...,...
245,Ziyu Yao,Assistant Professor\nComputer Science,N/A,Ziyu Yao works on natural language processing...,N/A,"PhD, Computer Science and Engineering, Ohio St...",mailto:ziyuyao@gmu.edu\n,https://cec.gmu.edu/node/6866
246,Grigory Yaroslavtsev,Assistant Professor\nComputer Science\nCollege...,N/A,Grigory Yaroslavtsev was previously an assist...,N/A,"PhD, Computer Science, Pennsylvania State Univ...",mailto:grigory@gmu.edu\n,https://cec.gmu.edu/node/6871
247,Jinwei Ye,Assistant Professor\nComputer Science\nCollege...,N/A,Jinwei Ye’s research interests include comput...,N/A,"PhD, Computer Science, University of Delaware ...",mailto:jinweiye@gmu.edu\n,https://cec.gmu.edu/node/6881
248,Binqian Yin,Assistant Professor,N/A,Binqian (Bella) Yin will join the Computer Sc...,N/A,"MS, Petroleum Engineering, University of Pitts...",,https://cec.gmu.edu/node/7926


In [70]:
#Data cleaning
#Remvoe "mailto: from beginning of email address and seperate email address from personal link"
df['Links'] = df['Links'].replace('mailto:','', regex=True)
new = df["Links"].str.split("\n", n = 1, expand = True)
df["Email"] = new[0]
df["Personal Link"] = new[1]
#Fill NaN and blanks with "N/A"
df.fillna("N/A",inplace=True)
df["Personal Link"] = df['Personal Link'].replace('','N/A', regex=True)
df["Biography"] = df['Biography'].replace('','N/A', regex=True)
df["Degrees"] = df['Degrees'].replace('','N/A', regex=True)
#Remove unnessary column
df.drop(columns =["Links"], inplace = True)
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",https://cec.gmu.edu/profiles/scarabal,,N/A
1,Omoche Cheche Agada,Assistant Professor / Teaching,N/A,Omoche Cheche Agada is an Assistant Professor...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,https://cec.gmu.edu/node/8046,N/A
2,Pouyan Ahmadi,Assistant Professor / Teaching,703-993-7155,Pouyan Ahmadi is an Assistant Professor in the...,N/A,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,,N/A
3,Massimiliano Albanese,Associate Professor / Tenured\nAssociate Chair...,N/A,I am an Associate Professor in the Department...,N/A,"PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,https://csis.gmu.edu/albanese/index.php,N/A
4,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",https://cec.gmu.edu/profiles/jallbeck,http://cs.gmu.edu/~jallbeck/,N/A
...,...,...,...,...,...,...,...,...,...
245,Ziyu Yao,Assistant Professor\nComputer Science,N/A,Ziyu Yao works on natural language processing...,N/A,"PhD, Computer Science and Engineering, Ohio St...",https://cec.gmu.edu/node/6866,ziyuyao@gmu.edu,N/A
246,Grigory Yaroslavtsev,Assistant Professor\nComputer Science\nCollege...,N/A,Grigory Yaroslavtsev was previously an assist...,N/A,"PhD, Computer Science, Pennsylvania State Univ...",https://cec.gmu.edu/node/6871,grigory@gmu.edu,N/A
247,Jinwei Ye,Assistant Professor\nComputer Science\nCollege...,N/A,Jinwei Ye’s research interests include comput...,N/A,"PhD, Computer Science, University of Delaware ...",https://cec.gmu.edu/node/6881,jinweiye@gmu.edu,N/A
248,Binqian Yin,Assistant Professor,N/A,Binqian (Bella) Yin will join the Computer Sc...,N/A,"MS, Petroleum Engineering, University of Pitts...",https://cec.gmu.edu/node/7926,,N/A


In [71]:
#Make sure email column only contains emails and if false, move link to personal link column
for i in range(0,len(df['Email'])):
    if "http" in df['Email'][i]:
        df['Personal Link'][i] = df['Email'][i]
        df['Email'][i] = "N/A"

In [72]:
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",https://cec.gmu.edu/profiles/scarabal,,N/A
1,Omoche Cheche Agada,Assistant Professor / Teaching,N/A,Omoche Cheche Agada is an Assistant Professor...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,N/A,https://cec.gmu.edu/node/8046
2,Pouyan Ahmadi,Assistant Professor / Teaching,703-993-7155,Pouyan Ahmadi is an Assistant Professor in the...,N/A,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,,N/A
3,Massimiliano Albanese,Associate Professor / Tenured\nAssociate Chair...,N/A,I am an Associate Professor in the Department...,N/A,"PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,N/A,https://csis.gmu.edu/albanese/index.php
4,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",https://cec.gmu.edu/profiles/jallbeck,N/A,http://cs.gmu.edu/~jallbeck/
...,...,...,...,...,...,...,...,...,...
245,Ziyu Yao,Assistant Professor\nComputer Science,N/A,Ziyu Yao works on natural language processing...,N/A,"PhD, Computer Science and Engineering, Ohio St...",https://cec.gmu.edu/node/6866,ziyuyao@gmu.edu,N/A
246,Grigory Yaroslavtsev,Assistant Professor\nComputer Science\nCollege...,N/A,Grigory Yaroslavtsev was previously an assist...,N/A,"PhD, Computer Science, Pennsylvania State Univ...",https://cec.gmu.edu/node/6871,grigory@gmu.edu,N/A
247,Jinwei Ye,Assistant Professor\nComputer Science\nCollege...,N/A,Jinwei Ye’s research interests include comput...,N/A,"PhD, Computer Science, University of Delaware ...",https://cec.gmu.edu/node/6881,jinweiye@gmu.edu,N/A
248,Binqian Yin,Assistant Professor,N/A,Binqian (Bella) Yin will join the Computer Sc...,N/A,"MS, Petroleum Engineering, University of Pitts...",https://cec.gmu.edu/node/7926,,N/A


In [73]:
df.to_csv('profiles_final_all.csv')

In [74]:
Department_Data = pd.read_csv('Individual_Department_Scrapped_Data.csv')

In [76]:
merged_df = df.merge(Department_Data, on=['Name'], how='outer', indicator=True)

new_data = merged_df[merged_df['_merge'] == 'right_only'][['Name','Title_y','Phone Number_y','Biography_y','Research Interest_y',
                                                          'Degrees_y','Profile Link_y','Email_y','Personal Link_y']]

new_data = new_data.rename(columns={'Title_y': 'Title', 'Phone Number_y': 'Phone Number', 'Biography_y': 'Biography',
                                   'Research Interest_y': 'Research Interest', 'Degrees_y': 'Degrees', 'Profile Link_y': 'Profile Link',
                                   'Email_y': 'Email', 'Personal Link_y': 'Personal Link'})

df = df.append(new_data, ignore_index=True)
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",https://cec.gmu.edu/profiles/scarabal,,N/A
1,Omoche Cheche Agada,Assistant Professor / Teaching,N/A,Omoche Cheche Agada is an Assistant Professor...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,N/A,https://cec.gmu.edu/node/8046
2,Pouyan Ahmadi,Assistant Professor / Teaching,703-993-7155,Pouyan Ahmadi is an Assistant Professor in the...,N/A,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,,N/A
3,Massimiliano Albanese,Associate Professor / Tenured\nAssociate Chair...,N/A,I am an Associate Professor in the Department...,N/A,"PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,N/A,https://csis.gmu.edu/albanese/index.php
4,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",https://cec.gmu.edu/profiles/jallbeck,N/A,http://cs.gmu.edu/~jallbeck/
...,...,...,...,...,...,...,...,...,...
260,Sai Manoj Pudukotai Dinakarrao,Assistant Professor,703-993-5083,"Sai Manoj Pudukotai Dinakarrao, an assistant ...",NaN,"PhD, Electrical and Electronic Engineering, N...",https://ece.gmu.edu/profiles/spudukot,spudukot@gmu.edu,http://mason.gmu.edu/~spudukot/\nhttps://orcid...
261,Shanjiang Zhu,Associate Professor,703-993-1797,Shanjiang Zhu is an associate professor of tra...,NaN,"PhD, Civil Engineering, University of Minneso...",https://civil.gmu.edu/profiles/szhu3,szhu3@gmu.edu,NaN
262,Syed Abbas Zaidi,"Professor, Systems Engineering and Operations ...",703-993-1774,"As an instructor, Abbas Zaidi’s pedagogical a...","Systems Architecture,","PhD, Information Technology, George Mason Uni...",https://seor.gmu.edu/profiles/szaidi2,NaN,http://seor.vse.gmu.edu/faculty/zaidi.html
263,Xuan Wang,Assistant Professor,NaN,Xuan Wang is an assistant professor in the De...,NaN,"PhD, Aeronautics and Astronautics, Purdue Univ...",https://ece.gmu.edu/profiles/xwang64,xwang64@gmu.edu,https://mason.gmu.edu/~xwang64/index.html\n


In [77]:
import numpy as np

Names_Mismatch = pd.read_csv('Name_Mismatch.csv')
df_Name = df.merge(Names_Mismatch, left_on='Name', right_on='Name_Scrapping', how='left')
df_Name.Name = np.where(df_Name.Name_Scrapping.isnull(), df_Name.Name, df_Name.Name_SpreadSheet)
df_Name = df_Name.iloc[:, [0,1,2,3,4,5,6,7,8]]
df_Name

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",https://cec.gmu.edu/profiles/scarabal,,N/A
1,Omoche Agada,Assistant Professor / Teaching,N/A,Omoche Cheche Agada is an Assistant Professor...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,N/A,https://cec.gmu.edu/node/8046
2,Pouyan Ahmadi,Assistant Professor / Teaching,703-993-7155,Pouyan Ahmadi is an Assistant Professor in the...,N/A,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,,N/A
3,Massimiliano Albanese,Associate Professor / Tenured\nAssociate Chair...,N/A,I am an Associate Professor in the Department...,N/A,"PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,N/A,https://csis.gmu.edu/albanese/index.php
4,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",https://cec.gmu.edu/profiles/jallbeck,N/A,http://cs.gmu.edu/~jallbeck/
...,...,...,...,...,...,...,...,...,...
260,Sai Manoj Pudukotai Dinakarrao,Assistant Professor,703-993-5083,"Sai Manoj Pudukotai Dinakarrao, an assistant ...",NaN,"PhD, Electrical and Electronic Engineering, N...",https://ece.gmu.edu/profiles/spudukot,spudukot@gmu.edu,http://mason.gmu.edu/~spudukot/\nhttps://orcid...
261,Shanjiang Zhu,Associate Professor,703-993-1797,Shanjiang Zhu is an associate professor of tra...,NaN,"PhD, Civil Engineering, University of Minneso...",https://civil.gmu.edu/profiles/szhu3,szhu3@gmu.edu,NaN
262,Syed Abbas Zaidi,"Professor, Systems Engineering and Operations ...",703-993-1774,"As an instructor, Abbas Zaidi’s pedagogical a...","Systems Architecture,","PhD, Information Technology, George Mason Uni...",https://seor.gmu.edu/profiles/szaidi2,NaN,http://seor.vse.gmu.edu/faculty/zaidi.html
263,Xuan Wang,Assistant Professor,NaN,Xuan Wang is an assistant professor in the De...,NaN,"PhD, Aeronautics and Astronautics, Purdue Univ...",https://ece.gmu.edu/profiles/xwang64,xwang64@gmu.edu,https://mason.gmu.edu/~xwang64/index.html\n


In [78]:
# Code added by Graphical Insights
# Reading the Names Given by CEC
CEC_Faculty_names = pd.read_csv('CEC_Faculty_List.csv')
#Left Join with CEC List on the left
df1 = CEC_Faculty_names.merge(df_Name, on='Name', how='left')
# For the New Names(Not in CEC Website) Taking the Job Title from the excel file
df1['Title'] = df1['Title'].fillna(df1['Job Title'])
#Replacing Null Values with N/A
df1 = df1.fillna('N/A')
# Selecting Required Columns 
df1 = df1.iloc[:, [0,6,7,8,9,10,11,12,13]]
df1

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Ariela Sofer,"Associate Dean, Administration and Faculty Aff...",703-993-1692,Ariela Sofer is interested in nonlinear progr...,"Optimization,","PhD, Operations Research, George Washington U...",https://cec.gmu.edu/profiles/asofer,N/A,http://mason.gmu.edu/~asofer/
1,Arthur Pyster,"Associate Dean for Research and Professor, Sys...",N/A,"Throughout his career, Art Pyster has played l...",N/A,"PhD, Computer and Information Sciences, The O...",https://cec.gmu.edu/profiles/apyster,apyster@gmu.edu,N/A
2,Deborah Goodings,"Professor and Associate Dean, College of Engin...",703-933-1504,Deborah Goodings is a professor and Associate...,"Geotechnical Engineering,","PhD, Geotechnical Engineering, Cambridge Univ...",https://cec.gmu.edu/profiles/goodings,,N/A
3,Sanjeev Setia,Professor\nComputer Science\nSchool of Computi...,703-993-4098,Sanjeev Setia is a professor of computer scien...,N/A,N/A,https://cec.gmu.edu/profiles/setia,setia@gmu.edu,http://cs.gmu.edu/~setia/\n
4,Giacomo Francisci,Postdoctoral Research Fellow,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...
301,Stone Bond,Sr Research Program Manager,N/A,N/A,N/A,N/A,N/A,N/A,N/A
302,Susan Schleigh,Senior Research Engineer,N/A,N/A,N/A,N/A,N/A,N/A,N/A
303,Traci Johnson,Research Manager,N/A,N/A,N/A,N/A,N/A,N/A,N/A
304,Joseph Kobsar,Sr Research Engineer,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [79]:
df1.to_csv('profiles_final_all.csv')